In [1]:
from abc import ABC, abstractmethod
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C as sb_A2C
from stable_baselines3 import PPO as sb_PPO
from stable_baselines3 import DDPG as sb_DDPG
from stable_baselines3 import TD3 as sb_TD3
from stable_baselines3.common.noise import ActionNoise
from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.type_aliases import Schedule
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as th
import yaml
import gym
from datetime import datetime
from sys import path
from os.path import dirname as dir
from typing import Any, Dict, Optional, Tuple, Type, Union
path.append(dir(path[0]))
print(path)
#__package__ = "examples"

from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer

['/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi/AgentLayer', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages/selenium-3.141.0-py3.8.egg', '/usr/local/lib/python3.8/site-packages/urllib3-1.26.4-py3.8.egg', '/Users/doganparlak/Desktop/Master_2.2/Master_Project/uniFi_github/uniFi']


In [2]:
class Environment(gym.Env, ABC):

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step(self, action):
        pass

    @abstractmethod
    def render(self, mode="human"):
        pass

    @abstractmethod
    def get_env(self):
        pass

    @staticmethod
    def softmax_normalization(actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output


In [3]:
class Agent(ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass
    
    @abstractmethod
    def load_model():
        pass

In [4]:
class ConventionalAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass
    
    @abstractmethod
    def _return_predict():
        pass

    @abstractmethod
    def _weight_optimization():
        pass

In [5]:
class RLAgent(Agent, ABC):

    @abstractmethod
    def train_model():
        pass

    @abstractmethod
    def predict():
        
        pass

    @abstractmethod
    def save_model():
        pass

    @abstractmethod
    def load_model():
        pass

In [6]:
class PortfolioEnv(Environment):

    def __init__(self,
                 df: pd.DataFrame,  # input data
                 stock_dim: int,  # number of unique securities in the investment universe
                 hmax: float,  # maximum number of shares to trade
                 initial_amount: float,  # initial cash value
                 transaction_cost_pct: float,  # transaction cost percentage per trade
                 reward_scaling: float,  # scaling factor for reward as training progresses
                 state_space: int,  # the dimension of input features (state space)
                 action_space: int,  # number of actions, which is equal to portfolio dimension
                 tech_indicator_list: list,  # a list of technical indicator names
                 turbulence_threshold=None,  # a threshold to control risk aversion
                 lookback=252,  #
                 day=0):  # an increment number to control date

        self.df = df
        self.day = day
        self.lookback = lookback
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(self.state_space + len(self.tech_indicator_list), self.state_space))
        

        # load data from a pandas dataframe
        
        ##FINRL APPROACH
        #self.df.set_index("date", drop = True, inplace=True)
        
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        #self.turbulence_threshold = turbulence_threshold
        # initalize state: initial portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
                
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            print(df)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
           # plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory, 'r')
           #plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, {}

        else:
            weights = Environment.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            # load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)         
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data["date"].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def render(self, mode='human'):
        return self.state

    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame({'date': date_list, 'daily_return': portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [7]:
class A2C(RLAgent):

    def __init__(self,
                 policy: "MlpPolicy",
                 env: None,
                 learning_rate: float = 7e-4,
                 n_steps: int = 5,
                 gamma: float = 0.99,
                 gae_lambda: float = 1.0,
                 ent_coef: float = 0.0,
                 vf_coef: float = 0.5,
                 max_grad_norm: float = 0.5,
                 rms_prop_eps: float = 1e-5,
                 use_rms_prop: bool = True,
                 use_sde: bool = False,
                 sde_sample_freq: int = -1,
                 normalize_advantage: bool = False,
                 tensorboard_log: Optional[str] = None,
                 create_eval_env: bool = False,
                 policy_kwargs: Optional[Dict[str, Any]] = None,
                 verbose: int = 0,
                 seed: Optional[int] = None,
                 device: Union[th.device, str] = "auto",
                 _init_setup_model: bool = True):

        self.env = env

        self.model = sb_A2C(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            gae_lambda= gae_lambda,
                            ent_coef = ent_coef,
                            vf_coef = vf_coef,
                            max_grad_norm = max_grad_norm,
                            rms_prop_eps= rms_prop_eps,
                            use_rms_prop= use_rms_prop,
                            use_sde= use_sde,
                            sde_sample_freq= sde_sample_freq,
                            normalize_advantage= normalize_advantage,
                            tensorboard_log=tensorboard_log,  
                            create_eval_env= create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device= device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):

        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]

    def save_model(self, path):
        self.model.save(path)

    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model

In [8]:
class PPO(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate:  3e-4,
                n_steps: int = 2048,
                batch_size: int = 64,
                n_epochs: int = 10,
                gamma: float = 0.99,
                gae_lambda: float = 0.95,
                clip_range: Union[float, Schedule] = 0.2,
                clip_range_vf: Union[None, float, Schedule] = None,
                normalize_advantage: bool = True,
                ent_coef: float = 0.0,
                vf_coef: float = 0.5,
                max_grad_norm: float = 0.5,
                use_sde: bool = False,
                sde_sample_freq: int = -1,
                target_kl: Optional[float] = None,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):

        self.env = env
        self.model = sb_PPO(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            n_steps = n_steps,
                            gamma = gamma,
                            batch_size = batch_size,
                            n_epochs = n_epochs,
                            gae_lambda=gae_lambda,
                            clip_range = clip_range,
                            clip_range_vf = clip_range_vf,
                            normalize_advantage=normalize_advantage,
                            ent_coef=ent_coef,
                            vf_coef=vf_coef,
                            max_grad_norm=max_grad_norm,
                            use_sde=use_sde,
                            sde_sample_freq=sde_sample_freq,
                            target_kl=target_kl,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)
    
    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model
    
    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
    
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [9]:
class DDPG(RLAgent):
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate : 1e-3,
                buffer_size: 1_000_000,  # 1e6
                learning_starts: 100,
                batch_size:  100,
                tau:  0.005,
                gamma:  0.99,
                train_freq:  1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_DDPG(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
        
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [28]:
class TD3(RLAgent):
 
    def __init__(self,
                policy: "MlpPolicy",
                env: None,
                learning_rate: float =  1e-3,
                buffer_size: int = 1_000_000,  # 1e6
                learning_starts: int = 100,
                batch_size: int = 100,
                tau: float = 0.005,
                gamma: float = 0.99,
                train_freq: int = 1,
                gradient_steps: int = -1,
                action_noise: Optional[ActionNoise] = None,
                replay_buffer_class: Optional[ReplayBuffer] = None,
                replay_buffer_kwargs: Optional[Dict[str, Any]] = None,
                optimize_memory_usage: bool = False,
                tensorboard_log: Optional[str] = None,
                create_eval_env: bool = False,
                policy_kwargs: Optional[Dict[str, Any]] = None,
                verbose: int = 0,
                seed: Optional[int] = None,
                device: Union[th.device, str] = "auto",
                _init_setup_model: bool = True):
                
        self.env = env
    
        self.model = sb_TD3(policy = policy,
                            env=self.env,
                            learning_rate = learning_rate,
                            buffer_size = buffer_size,
                            learning_starts= learning_starts,
                            batch_size = batch_size,
                            tau = tau,
                            gamma= gamma,
                            train_freq = train_freq,
                            gradient_steps = gradient_steps,
                            action_noise= action_noise,
                            replay_buffer_class= replay_buffer_class,
                            replay_buffer_kwargs= replay_buffer_kwargs,
                            optimize_memory_usage=optimize_memory_usage,
                            tensorboard_log=tensorboard_log,
                            create_eval_env=create_eval_env,
                            policy_kwargs=policy_kwargs,
                            verbose=verbose,
                            seed=seed,
                            device=device,
                            _init_setup_model = _init_setup_model)

    def train_model(self, **train_params):
        self.model = self.model.learn(**train_params)
        return self.model

    def predict(self, environment, **test_params):
        env_test, obs_test = environment.get_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []

        env_test.reset()
        for i in range(len(environment.df.index.unique())):
            action, _states = self.model.predict(obs_test, **test_params)
            obs_test, rewards, dones, info = env_test.step(action)
            if i == (len(environment.df.index.unique()) - 2):
                account_memory = env_test.env_method(method_name="save_asset_memory")
                actions_memory = env_test.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break

        return account_memory[0], actions_memory[0]
        
    def load_model(self, path):
        self.model = self.model.load(path)
        return self.model
    
    def save_model(self, path):
        self.model.save(path)

In [13]:
#Gather user parameters
with open("../user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [14]:
tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]
train_params = user_params["train_params"]
policy_params = user_params["policy_params"]
test_params = user_params["test_params"]

In [15]:
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                                end_date='2021-10-31',
                                ticker_list= tickers).download_from_yahoo()
print(downloaded_df.head())


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)
        date       open       high        low      close     volume   tic  day
0 2008-12-31   3.070357   3.133571   3.047857   2.606278  607541200  AAPL    2
1 2008-12-31  41.590000  43.049999  41.500000  32.005886    5443100    BA    2
2 2008-12-31  43.700001  45.099998  43.700001  30.628826    6277400   CAT    2
3 2008-12-31  72.900002  74.629997  72.900002  43.670765    9964300   CVX    2
4 2009-01-02   3.067143   3.251429   3.041429   2.771174  746015200  AAPL    4


In [16]:
print("\nTest 4: Feature engineer.........")

df_processed = DefaultFeatureEngineer( use_default= False,
                                       tech_indicator_list= env_kwargs["tech_indicator_list"],
                                       use_vix=True,
                                       use_turbulence=True,
                                       use_covar=True).extend_data(downloaded_df)  # included technical indicators as features

print(df_processed.head())


Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances
        date  ...                                        return_list
0 2009-12-31  ...  tic             AAPL        BA       CAT      ...
0 2009-12-31  ...  tic             AAPL        BA       CAT      ...
0 2009-12-31  ...  tic             AAPL        BA       CAT      ...
0 2009-12-31  ...  tic             AAPL        BA       CAT      ...
1 2010-01-04  ...  tic             AAPL        BA       CAT      ...

[5 rows x 20 columns]


In [17]:
env = PortfolioEnv(df=df_processed, **env_kwargs) 

In [18]:
env_train, _ = env.get_env()

In [19]:
td3 = TD3(env = env_train, **policy_params["TD3_PARAMS"])

In [20]:
td3.train_model(**train_params["TD3_PARAMS"])

In [21]:
env_test = PortfolioEnv(df=df_processed, **env_kwargs) 

In [22]:
td3.predict(environment = env_test, **test_params["TD3_PARAMS"])

             0
0     0.000000
1     0.024403
2     0.010359
3    -0.000608
4     0.005737
...        ...
2973 -0.002207
2974  0.003908
2975 -0.004611
2976 -0.012020
2977  0.026037

[2978 rows x 1 columns]
begin_total_asset:1000000
end_total_asset:9641012.870825065
Sharpe:  0.9484473772200268
hit end!


(           date  daily_return
 0    2009-12-31      0.000000
 1    2010-01-04      0.024403
 2    2010-01-05      0.010359
 3    2010-01-06     -0.000608
 4    2010-01-07      0.005737
 ...         ...           ...
 2973 2021-10-22     -0.002207
 2974 2021-10-25      0.003908
 2975 2021-10-26     -0.004611
 2976 2021-10-27     -0.012020
 2977 2021-10-28      0.026037
 
 [2978 rows x 2 columns],
                 AAPL        BA       CAT       CVX
 date                                              
 2009-12-31  0.250000  0.250000  0.250000  0.250000
 2010-01-04  0.365529  0.134471  0.365529  0.134471
 2010-01-05  0.365529  0.134471  0.365529  0.134471
 2010-01-06  0.365529  0.134471  0.365529  0.134471
 2010-01-07  0.365529  0.134471  0.365529  0.134471
 ...              ...       ...       ...       ...
 2021-10-22  0.365529  0.134471  0.365529  0.134471
 2021-10-25  0.365529  0.134471  0.365529  0.134471
 2021-10-26  0.365529  0.134471  0.365529  0.134471
 2021-10-27  0.365529  0.134

In [23]:
td3.save_model("AgentLayer/RLAgents/td3_model")

/usr/local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'AgentLayer/RLAgents' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [26]:
loaded_td3_model = td3.load_model("AgentLayer/RLAgents/td3_model")

In [27]:
loaded_td3_model